In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from tqdm import tqdm 
import os
import h5py

In [ ]:
class PendulumSimulator:
    # simulator for pendulum systems using pytorch for differentiable physics
    def __init__(self, device="mps"): # change device accordingly
        # initialize the pendulum simulator
        # Args: device(str): Device to run compuatations on
        self.device = device
        self.g = 9.81 # acceleration due to gravity (m/s^2)

        # default params of the pendulum
        self.L = 1.0 # lenght of pendulum string (m)
        self.m = 1.0 # mass of bob (kg)

    
    def set_params(self, **kwargs):
        # set the parameters for the pendulum system
        for key, value in kwargs.items():
            if hasattr(self, key):
                setattr(self, key, value)
            else:
                raise ValueError(f"parameter '{key}' invalid.")
            
    # compute the derivatives for a simple pendulum
    def dynamics(self, state, t=0):
        '''
        Arguments: state(torch.Tensor): State vector [theta, omega]: angle and velocity
                   t (float): Time
        Returns:
                torch.Tensor: Derivatives [dTheta/dt, dOmega/dt]

        '''
        theta, omega = state[..., 0:1], state[..., 1:2]

        # compute the derivatives
        dTheta_dt=omega
        dOmega_dt = -(self.g/self.L)*torch.sin(theta)

        return torch.cat([dTheta_dt, dOmega_dt], dim=-1)


    # perform a single step of runge-kutta 4 method
    def rk4_step(self, state, dt):
        k1 = self.dynamics(state)
        k2 = self.dynamics(state + 0.5*dt*k1)
        k3 = self.dynamics(state + 0.5*dt*k2)
        k4 = self.dynamics(state + dt*k3)

        return state+(dt/6.0)*(k1 + 2 * k2 + 2 * k3 + k4)


    # simulate the system form a given initial state
    def simulate(self, initial_state, duration, dt=0.01, return_tensor=True):
        '''
        Arguments:
            initial_state (torch.Tensor or list): Initial state vector
            duration (float): duration of the sim in seconds
            dt (fl): time step for numerical intergation
            return_tensor (bool): True: return a tensor, False: return  NumPy array

        Returns:
            torch.Tensor or numpy.ndarry: the trajectory of the states
        '''
        # covnerting init state to tensor if required
        if not isinstance(initial_state, torch.Tensor):
            initial_state = torch.Tensor(initial_state, dtype=torch.float32, device=self.device)

        
        # make sure the init state has the correct shape
        if initial_state.dim() ==1:
            initial_state = initial_state.unsqueeze(0)

        # number of steps
        n_steps = int(duration / dt)

        # initialize the trajectory tensor
        trajectory = torch.zeros((n_steps + 1, *initial_state.shape), device=self.device)

        # time steps
        times = torch.linspace(0, duration, n_steps+1, device=self.device)

        # simulating forwards in time
        state=initial_state
        for i in range(n_steps):
            state=self.rk4_step(state,dt)
            trajectory[i+1] = state

        if return_tensor:
            return times, trajectory
        else:
            return times.cpu().numpy(), trajectory.cpu().numpy()
        

    # computing the cartesian coords from state
    def compute_cartesian_coords(self, states):
        ''' 
        args:
            states(torch.Tensor): state vector(s) of the pendulum
        return:
            tuple: (x, y) coords as pytorch tensors
        '''
        x = self.L*torch.sin(states[..., 0])
        y = self.L*torch.cos(states[..., 0])

        return x, y


    # compute the total energy of the system
    def compute_energy(self, states):
        '''
        arguments:
            states(torch.Tensor): state vector(s) of the pendulum

        return:
            torch.Tensor: total energy
        '''

        theta, omega = states[..., 0], states[..., 1]

        # KE
        T = 0.5 * self.m * (self.L * omega)**2

        # PE
        U = self.m * self.g * self.L(1-torch.cos(theta))

        # TE
        E = T+U

        return E

        
    
